#NB Classifier

In [1]:
import pandas as pd
import csv

In [17]:
df = pd.read_csv("/content/drive/MyDrive/Truth_Seeker/cleaned_master_data.csv")

In [19]:
df

,title,lemmatized_text,label,subject,text_length,title_length
0,\r\r\r\r\r\r\r\nDonald Trump Gets His A Handed...,yearold girl smacked donald trump amazingon sa...,0,General News,2566,79
1,\r\r\r\r\r\r\r\nExGOP Congressman Shreds Fello...,point every republican refuse spine stand dona...,0,General News,1925,104
2,\r\r\r\r\r\r\r\nTrump Gets STOMPED For Whining...,donald trump attacking president obama campaig...,0,General News,3389,106
3,\r\r\r\r\r\r\r\nWATCH Democratic Rep Delivers ...,massachusetts rep jim mcgovern brought house s...,0,General News,3358,117
4,AfterTrumpImplodes Hashtag Hilariously Imagin...,world like postdonald trump new hashtag aftert...,0,General News,3092,86
...,...,...,...,...,...,...
75770,white coat summit dr 99 8 percent of people ge...,percent people get little progressive signific...,0,Unknown,1812,90
75771,who the hel elected you dorsey admits theres n...,twitter facebook google running protection rac...,0,Unknown,8215,91
75772,woke business list still demanding masks in fr...,one new coowners privateequity firm engulfed c...,0,Unknown,2190,54
75773,you spent a fing week on pneumoniagate,people perceive womens health energy level cam...,1,Unknown,14102,38


In [20]:
df.shape

(75775, 6)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75775 entries, 0 to 75774
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            75775 non-null  object
 1   lemmatized_text  75775 non-null  object
 2   label            75775 non-null  int64 
 3   subject          75775 non-null  object
 4   text_length      75775 non-null  int64 
 5   title_length     75775 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 3.5+ MB


In [22]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

In [23]:
df_filtered = df[['lemmatized_text', 'label']]

In [24]:
X= df_filtered['lemmatized_text']
y = df_filtered['label']

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [26]:
vectorizer = CountVectorizer(stop_words='english')
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [27]:
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_vec, y_train)

MultinomialNB()

In [28]:
y_pred = nb_classifier.predict(X_test_vec)

In [29]:
nb = accuracy_score(y_test, y_pred)*100

In [30]:
print("Accuracy:", nb)
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 81.13493896403827

Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.81      0.78      6419
           1       0.85      0.82      0.83      8736

    accuracy                           0.81     15155
   macro avg       0.81      0.81      0.81     15155
weighted avg       0.81      0.81      0.81     15155



In [ ]:
doc = 'In a groundbreaking discovery, a group of scientists has reportedly found a cure for cancer that could save millions of lives. However, sources within the research community claim that the cure is being suppressed by powerful pharmaceutical companies who fear the loss of profits from cancer treatments. Despite growing support for the findings, the medical industry and government officials are said to be working together to keep the discovery under wraps.There\'s a cure for cancer, and it\’s been available for years, said an anonymous scientist who claims to have worked on the treatment. But the big drug companies don’t want the world to know about it because they\’d lose billions of dollars in revenue from chemotherapy drugs and radiation treatments.Medical experts around the world have cast doubt on these claims, calling it a conspiracy theory with no scientific backing. Yet, the story continues to circulate in alternative media outlets and among conspiracy theorists. Critics argue that the narrative is a part of a larger agenda to control the health sector and profit from people\'s suffering.'

In [ ]:
doc1 = "OpenAI recently launched GPTs, which allow users to create custom versions of its AI models tailored to specific tasks. This feature empowers businesses and individuals to build more personalized and efficient AI systems without requiring deep technical expertise. Alongside this, OpenAI introduced the ChatGPT Enterprise subscription, which includes robust data encryption and advanced analytics tools."

In [ ]:
doc1 = vectorizer.transform([doc1])

In [ ]:
nb_classifier.predict(doc1)

array([1])

In [31]:
from sklearn.model_selection import GridSearchCV

param_grid = {'alpha': [0.0001, 0.5, 1, 2, 5, 10]}

grid_search = GridSearchCV(estimator=MultinomialNB(),param_grid=param_grid,scoring='accuracy',cv=5,verbose=1)

grid_search.fit(X_train_vec, y_train)

print("Best Cross-Validation Accuracy:", grid_search.best_score_*100)

best_model = grid_search.best_estimator_
y_pred_tuned = best_model.predict(X_test_vec)

tuned = accuracy_score(y_test, y_pred_tuned)*100
print("\nTuned Accuracy:", tuned)
print("\nClassification Report(Tuned):\n", classification_report(y_test, y_pred_tuned))

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Best Cross-Validation Accuracy: 82.53876608380072

Tuned Accuracy: 82.81095348069944

Classification Report(Tuned):
               precision    recall  f1-score   support

           0       0.78      0.82      0.80      6419
           1       0.86      0.83      0.85      8736

    accuracy                           0.83     15155
   macro avg       0.82      0.83      0.83     15155
weighted avg       0.83      0.83      0.83     15155



#SVM

In [32]:
from sklearn.svm import SVC

In [33]:
svm_classifier = SVC(class_weight='balanced')

In [34]:
df_sample = df_filtered.sample(n=1000, random_state=42)

In [35]:
X = df_sample['lemmatized_text']
y = df_sample['label']

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [37]:
vectorizer = CountVectorizer(stop_words='english')
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [38]:
svm_classifier.fit(X_train_vec, y_train)

SVC(class_weight='balanced')

In [39]:
y_pred = svm_classifier.predict(X_test_vec)

In [40]:
svm = accuracy_score(y_test, y_pred)*100

In [41]:
print("Accuracy: ", svm)
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy:  74.5

Classification Report:
               precision    recall  f1-score   support

           0       0.70      0.77      0.73        91
           1       0.79      0.72      0.76       109

    accuracy                           0.74       200
   macro avg       0.74      0.75      0.74       200
weighted avg       0.75      0.74      0.75       200



In [42]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 1.5, 0.5],'kernel': ['linear', 'rbf'],'gamma': ['scale', 'auto']}

grid_search = GridSearchCV(svm_classifier, param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train_vec, y_train)

print("Best parameters found: ", grid_search.best_params_)

best_svm_classifier = grid_search.best_estimator_

Best parameters found:  {'C': 1.5, 'gamma': 'scale', 'kernel': 'rbf'}


In [43]:
svm_tuned = accuracy_score(best_svm_classifier.predict(X_test_vec), y_test)*100

In [44]:
print("Accuracy:",svm_tuned)

Accuracy: 77.5


In [46]:
print("NB Accuracy:",tuned)
print("SVM Accuracy:",svm_tuned)

NB Accuracy: 82.81095348069944
SVM Accuracy: 77.5
